In [ ]:
!pip install keras-tuner        #library which is is used to find out the number of hidden layers and the number of neurons in each hidden layer.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch     #this decides the number of hidden layers and number of neurons in a hidden layer from a provided range of numbers.

In [ ]:
df = pd.read_csv("Real_Combine.csv")

In [ ]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [ ]:
x = df.iloc[:, :-1]
y = df.iloc[:, 1]

In [ ]:
def build_model(hp):
  model = keras.Sequential()
  for i in range(hp.Int("num_layers", 2, 20)):      #specifying that neural network can have hidden layers between 2 to 20, the tuner will test different numbers
    model.add(layers.Dense(units = hp.Int("units_" + str(i),
                                          min_value = 32,     #min number of neurons in a hidden layer
                                          max_value = 512,    #max number of neurons in a hidden layer
                                          step = 32),         #32, 64, 96, 128
                           activation = 'relu'))
    model.add(layers.Dense(1, activation = 'linear'))
    model.compile(
        optimizer = keras.optimizers.Adam(
            hp.Choice("learning_rate",[1e-2, 1e-3, 1e-4])     #learning rate is a hyperparameter  and the range is 0.001, 0.0001, 0.00001
        ),
        loss = "mean_absolute_error",
        metrics = ["mean_absolute_error"]                     #this is the performance metric tracked by the model during the training
    )
    return model

#hp.Int = model tries different values from the given range
#hp.Choice = model chooses from given options

In [ ]:
tuner =  RandomSearch(
    build_model,
    objective = "val_mean_absolute_error",      #this objective is optimized during training; for classification problems, this will be val_accuracy
    max_trials = 5,                             #the number of combinations tuner will try
    executions_per_trial = 3,                   #number of times each combination is executed
    directory = "project1",                      #all the weights are stored here
    project_name = "Air Quality Index"
)

Reloading Tuner from project1/Air Quality Index/tuner0.json


In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
tuner.search(xtrain, ytrain, epochs = 5, validation_data = (xtest, ytest))

In [ ]:
tuner.results_summary()

Results summary
Results in project1/Air Quality Index
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 1 summary
Hyperparameters:
num_layers: 2
units_0: 384
learning_rate: 0.001
Score: 1.5119138558705647

Trial 4 summary
Hyperparameters:
num_layers: 20
units_0: 384
learning_rate: 0.001
Score: 1.5389164686203003

Trial 3 summary
Hyperparameters:
num_layers: 18
units_0: 96
learning_rate: 0.01
Score: 1.9632048606872559

Trial 0 summary
Hyperparameters:
num_layers: 3
units_0: 320
learning_rate: 0.01
Score: 2.899717370669047

Trial 2 summary
Hyperparameters:
num_layers: 17
units_0: 160
learning_rate: 0.0001
Score: 6.830006758371989
